In [ ]:
from scipy.stats.stats import pearsonr
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/Users/desert/desert_workspace/final_data.csv')
df=df.drop('Unnamed: 0', axis=1)
df.head()


In [ ]:
data=df[df.columns.tolist()[2:40]]

In [ ]:
# calculate the correlation matrix
corr_dataframe = data.corr()

# compute hierarchical cluster on both rows and columns for correlation matrix and plot heatmap 
def corr_heatmap(corr_dataframe):
    import scipy.cluster.hierarchy as sch
    
    corr_matrix = np.array(corr_dataframe)
    col_names = corr_dataframe.columns
    
    Y = sch.linkage(corr_matrix, 'single', 'correlation')
    Z = sch.dendrogram(Y, color_threshold=0, no_plot=True)['leaves']
    corr_matrix = corr_matrix[Z, :]
    corr_matrix = corr_matrix[:, Z]
    col_names = col_names[Z]
    im = plt.imshow(corr_matrix, interpolation='nearest', aspect='auto', cmap='bwr')
    plt.colorbar()
    plt.xticks(range(corr_matrix.shape[0]), col_names, rotation='vertical', fontsize=4)
    plt.yticks(range(corr_matrix.shape[0]), col_names[::-1], fontsize=4)
    
# plot
corr_heatmap(corr_dataframe)

In [ ]:

def remove_high_corr(corr_dataframe, thresh = 0.9):
    '''remove predictors with high pairwise correlation'''
    abs_corr = np.abs(corr_dataframe).as_matrix() # absolute correlation matrix
    col_names = list(corr_dataframe.columns)
    
    # set up diagonal to 0
    np.fill_diagonal(abs_corr, 0)
    
    print "Removed predictors (in order): \n"
    while np.max(abs_corr) >= thresh:
        i, j = np.unravel_index(abs_corr.argmax(), abs_corr.shape) # find maximum element
        # print abs_corr[i, j]
        rdx = which_to_remove(i, j, abs_corr)
        # remove corresponding predictor
        print col_names.pop(rdx)
        abs_corr = np.delete(abs_corr, rdx, 0)
        abs_corr = np.delete(abs_corr, rdx, 1)
        
    return col_names

def which_to_remove(i, j, abs_corr):
    '''compare two predictors and remove the one with higher abs correlation with other predictors'''
    i_absmean = np.mean(abs_corr[i, np.where(abs_corr[i,:] == 0)])
    j_absmean = np.mean(abs_corr[j, np.where(abs_corr[j,:] == 0)])
    
    return i if i_absmean > j_absmean else j

# remained predictors
col_remained = remove_high_corr(corr_dataframe)
data=data[col_remained]
corr_dataframe = data.corr()

corr_heatmap(corr_dataframe)

In [ ]:
col_remained = remove_high_corr(corr_dataframe)
data = df[col_remained]
corr_dataframe = data.corr()

corr_heatmap(corr_dataframe)

In [ ]:
corrmat = data.corr()

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(16, 12))

# Draw the heatmap using seaborn, and add a title to the plot
sns.heatmap(corrmat, vmax=.8, square=True)
ax.set_title('CA Food Desert Data Correlations')
f.tight_layout()

In [ ]:
data.columns.tolist()


model = forward_selected(data, 'pop2010_in_des')

print model.model.formula

print model.rsquared_adj

In [ ]:
model.summary()

In [ ]:

from bokeh.charts import Histogram
from bokeh.charts import defaults, vplot, hplot, show, output_file
from bokeh.io import output_notebook 
output_notebook()


# input options
hist = Histogram(df['pop2010_in_des'], title="df['pop2010_in_des']")
show(hist)
#df['pop2010_in_des']

In [ ]:
sns.distplot(df['percent_food_desert'])

In [ ]:
mean_expected_value = df['percent_food_desert'].mean()
Squared_errors = pd.Series(mean_expected_value - df['percent_food_desert'])**2
SSE = np.sum(Squared_errors)
print ('Sum of Squared Errors (SSE): %01.f' % SSE)
density_plot = Squared_errors.plot('hist')


The plot shows how frequent certain errors are in respect of their values. Therefore, we can see that most errors are around zero (there is a high density around that value). Such a situation can be considered a good one, since in most cases the mean is a good approximation, but some errors are really very far from the zero and they can attain considerable values (don't forget that the errors are squared, anyway, so the effect is emphasized). When trying to  gure out such values, our approach will surely lead to a relevant error and we should find a way to minimize it using a more sophisticated approach.

Evidently, the mean is not a good representative of certain values, but it is certainly a good baseline to start from. Certainly, an important problem with the mean is
its being  xed, whereas the target variable is changeable. However, if we assume that the target variable changes because of the effect of some other variable we are measuring, then we can adjust the mean with respect to the variations in cause.

One improvement on our previous approach could be to build a mean conditional on certain values of another variable (or even more than one) actually related to our target, whose variation is somehow similar to the variation of the target one.  

Intuitively, if we know the dynamics we want to predict with our model, we can try to look for variables that we know can impact the answer values.   

In food deserts, we actually know that usually the larger a county population is,
the more food desert census tracts it will posess; however, this rule is just part of the story and the number of food deserts possess by a county is affected by many other considerations. For the moment, we will keep it simple and just assume that the unemployment rate is a factor that negatively affects a counties well being, and consequently, results in more food deserts.  

Now, we have a variable that we know should change with our target and we just need to measure it and extend our initial formula based on constant values with something else.
In statistics, there is a measure that helps to measure how (in the sense of how much and in what direction) two variables relate to each other: 
#### correlation. ####
In correlation, a few steps are to be considered. First, your variables have to be standardized (or your result won't be a correlation but a covariation, a measure of association that is affected by the scale of the variables you are working with).

In statistical Z score standardization, you subtract from each variable its mean and then you divide the result by the standard deviation. The resulting transformed variable will have a mean of 0 and a standard deviation of 1 (or unit variance, since variance is the squared standard deviation).
The formula for standardizing a variable is as follows:

$$x = x-mean(x) / std(x)$$


After standardizing, you compare the squared difference of each variable with its own mean. If the two differences agree in sign, their multiplication will become positive (evidence that they have the same directionality); however, if they differ, the multiplication will turn negative. By summing all the multiplications between the squared differences, and dividing them by the number of observations, you will  finally get the correlation which will be a number ranging from -1 to 1.
The absolute value of the correlation will provide you with the intensity of the relation between the two variables compared, 1 being a sign of a perfect match and zero a sign of complete independence between them (they have no relation between them). The sign instead will hint at the proportionality; positive is direct (when one grows the other does the same), negative is indirect (when one grows, the other shrinks).


Covariance can be expressed as follows:

$$ cov(xi,y) = 1/n * sum(xi - mean(x1) * (y-mean(y)) $$

Whereas, Pearson's correlation can be expressed as follows:

$$ r = 1/n * {sum(xi - mean(x1) * (y-mean(y)) / std(xi * std(y)} $$

In [ ]:
def covariance(variable_1, variable_2, bias=0):
    observations = float(len(variable_1))
    return np.sum((variable_1 - np.mean(variable_1)) * (variable_2 - np.mean(variable_2)))/(observations-min(bias,1))

def standardize(variable):
    return (variable - np.mean(variable)) / np.std(variable)

def correlation(var1,var2,bias=0):
    return covariance(standardize(var1), standardize(var2),bias)

In [ ]:
from scipy.stats.stats import pearsonr
print ('Our correlation estimation: %0.5f' % (correlation(df['unemployment_rate'], df['percent_food_desert'])))
print ('Correlation from Scipy pearsonr estimation: %0.5f' % pearsonr(df['unemployment_rate'], df['percent_food_desert'])[0])


Our correlation estimation for the relation between the value of the target variable and the county unemployment rate in the area is 0.45024, which is positive and considerably strong, since the maximum positive score of a correlation is 1.0.

In [ ]:
x_range = [df['unemployment_rate'].min(),df['unemployment_rate'].max()]
y_range = [df['percent_food_desert'].min(),df['percent_food_desert'].max()]
scatter_plot = df.plot(kind='scatter', x='unemployment_rate', y='percent_food_desert', xlim=x_range, ylim=y_range)
meanY = scatter_plot.plot(x_range, [df['percent_food_desert'].mean(),df['percent_food_desert'].mean()], '--' , color='red', linewidth=1)
meanX = scatter_plot.plot([df['unemployment_rate'].mean(),df['unemployment_rate'].mean()], y_range, '--', color='red', linewidth=1)


In [ ]:
y = df['percent_food_desert']
X = df['unemployment_rate']
linear_regression = smf.ols(formula='percent_food_desert ~ unemployment_rate', data=df)
fitted_model = linear_regression.fit()
fitted_model.summary()


In [ ]:
print (fitted_model.params)
betas = np.array(fitted_model.params)
fitted_values = fitted_model.predict()

• Dep. Variable: It just reminds you what the target variable was  
• Model: Another reminder of the model that you have fitted, the OLS is
ordinary least squares, another way to refer to linear regression   
• Method: The parameters fitting method (in this case least squares, the classical computation method)   
• No. Observations: The number of observations that have been used   
• DF Residuals: The degrees of freedom of the residuals, which is the number
of observations minus the number of parameters     
• DF Model: The number of estimated parameters in the model (excluding the constant term from the count)  

The second table gives a more interesting picture, focusing how good the  t of the linear regression model is and pointing out any possible problems with the model:  
  
• R-squared: This is the coefficient of determination, a measure of how well the regression does with respect to a simple mean.  
• Adj. R-squared: This is the coefficient of determination adjusted based on the number of parameters in a model and the number of observations that helped build it.   
• F-statistic: This is a measure telling you if, from a statistical point of view, all your coefficients, apart from the bias and taken together, are different from zero. In simple words, it tells you if your regression is really better than a simple average.   
• Prob (F-statistic): This is the probability that you got that F-statistic just by lucky chance due to the observations that you have used (such a probability is actually called the p-value of F-statistic). If it is low enough you can be confident that your regression is really better than a simple mean. Usually in statistics and science a test probability has to be equal or lower than 0.05    
(a conventional criterion of statistical significance) for having such a confidence.
• AIC: This is the Akaike Information Criterion. AIC is a score that evaluates the model based on the number of observations and the complexity of
the model itself. The lesser the AIC score, the better. It is very useful for comparing different models and for statistical variable selection.   
• BIC: This is the Bayesian Information Criterion. It works as AIC, but it presents a higher penalty for models with more parameters.   

Most of these statistics make sense when we are dealing with more than one predictor variable, so they will be discussed in the next notebook. Thus, for the moment, as we are working with a simple linear regression, the two measures that are worth examining closely are F-statistic and R-squared.
_______
* F-statistic is actually a test that doesn't tell you too much if you have enough observations and you can count on a minimally correlated predictor variable. Usually it shouldn't be much of a concern in a data science project.  

* R-squared is instead much more interesting because it tells you how much better your regression model is in comparison to a single mean. It does so by providing you with a percentage of the unexplained variance of a mean as a predictor that actually your model was able to explain.

If you want to compute the measure yourself, you just have to calculate the sum of squared errors of the mean of the target variable. That's your baseline of unexplained variance (the variability in the percentage of population living in a food desert that in our example we want to explain by a model). If from that baseline you subtract the sum of squared errors of your regression model, you will get the residual sum of squared errors, which can be compared using a division with your baseline:

In [ ]:
mean_sum_squared_errors = np.sum((df['percent_food_desert']-df['percent_food_desert'].mean())**2)
regr_sum_squared_errors = np.sum((df['percent_food_desert']-fitted_values)**2)
(mean_sum_squared_errors-regr_sum_squared_errors) / mean_sum_squared_errors

In [ ]:
mean_sum_squared_errors

In [ ]:
regr_sum_squared_errors

The second output table informs us about the coef cients and provides us with a series of tests. These tests can make us con dent that we have not been fooled by a few extreme observations in the foundations of our analysis or by some other problem:
• coef: The estimated coefficient  
• std err: The standard error of the estimate of the coefficient; the larger it is,
the more uncertain the estimation of the coefficient  
• t: The t-statistic value, a measure indicating whether the coefficient true value is different from zero  
• P > |t|: The p-value indicating the probability that the coefficient is different from zero just by chance  
• [95.0% Conf. Interval]: The lower and upper values of the coefficient, considering 95% of all the chances of having different observations and so different estimated coefficients  

$$ y = -0.1220 * x_{unemploymentrate} * 0.0260$$

In [ ]:
# Folmula applied to Alameda county
-0.1220 * 8.208532 * 0.0260
# Close to 0.037134?

a linear regression can always work within the range of values it learned from (this is called interpolation) but can provide correct values for its learning boundaries
(a different predictive activity called extrapolation) only in certain conditions.

Standard errors instead are very important because they signal a weak or unclear relationship between the predictor and the answer. You can notice this by dividing the standard error by its beta. If the ratio is 0.5 or even larger, then it's a clear sign that the model has little con dence that it provided you with the right coef cient estimates. Having more cases is always the solution because it can reduce the standard errors of the coef cients and improve our estimates; however, there are also other methods to reduce errors, such as removing the redundant variance present among the features by a principal component analysis or selecting a parsimonious set of predictors by greedy selections. All these topics will be discussed when we work with multiple predictors; at this point in the book, we will illustrate the remedies to such a problem.

In [ ]:
#regr_sum_squared_errors/ betas
fitted_model.summary()

The last table deals with an analysis of the residuals of the regression. The residuals are the difference between the target values and the predicted  tted values:
  
• Skewness: This is a measure of the symmetry of the residuals around the mean. For symmetric distributed residuals, the value should be around zero. A positive value indicates a long tail to the right; a negative value a long tail to the left.  
• Kurtosis: This is a measure of the shape of the distribution of the residuals. A bell-shaped distribution has a zero measure. A negative value points to a too flat distribution; a positive one has too great a peak.  
• Omnibus D'Angostino's test: This is a combined statistical test for skewness and kurtosis.  
• Prob(Omnibus): This is the Omnibus statistic turned into a probability.  
• Jarque-Bera: This is another test of skewness and kurtosis.  
• Prob (JB): This is the JB statistic turned into a probability.  
• Durbin-Watson: This is a test for the presence of correlation among the residuals (relevant during analysis of time-based data).  
• Cond. No: This is a test for multicollinearity (we will deal with the concept of multicollinearity when working with many predictors).  

In [ ]:
fitted_model.summary()

In [ ]:
residuals = df['percent_food_desert']-fitted_values
normalized_residuals = standardize(residuals)
residual_scatter_plot = plt.plot(df['unemployment_rate'], normalized_residuals,'bp')
mean_residual = plt.plot([int(x_range[0]),round(x_range[1],0)], [0,0],'-', color='red', linewidth=2)
upper_bound = plt.plot([int(x_range[0]),round(x_range[1],0)], [3,3], '--', color='red', linewidth=1)
lower_bound = plt.plot([int(x_range[0]),round(x_range[1],0)], [-3,-3],'--', color='red', linewidth=1)
plt.grid()

In [ ]:
df.plot(kind='scatter', x='unemployment_rate', y='percent_food_desert')
plt.plot(pd.DataFrame(df['unemployment_rate']),fitted_values,c='red',linewidth=2)


In [ ]:
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_partregress_grid(fitted_model, fig=fig)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
fig = sm.graphics.plot_fit(fitted_model, "unemployment_rate", ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
fig = sm.graphics.plot_leverage_resid2(fitted_model, ax=ax)

In [ ]:
fig = plt.figure(figsize=(12, 8))
fig = sm.graphics.plot_ccpr_grid(fitted_model, fig=fig)

Minimizing the Cost Function
---
At the core of linear regression, there is the search for a line's equation that it is able to minimize the sum of the squared errors of the difference between the line's y values and the original ones.

In [ ]:
def squared_cost(v,e):
    return np.sum((v-e)**2)

In [ ]:
import random

def random_w( p ):
    return np.array([np.random.normal() for j in range(p)])

def hypothesis(X,w):
    return np.dot(X,w)

def loss(X,w,y):
    return hypothesis(X,w) - y

def squared_loss(X,w,y):
    return loss(X,w,y)**2

def gradient(X,w,y):
    gradients = list()
    n = float(len( y ))
    for j in range(len(w)):
        gradients.append(np.sum(loss(X,w,y) * X[:,j]) / n)
    return gradients

def update(X,w,y, alpha=0.01):
    return [t - alpha*g for t, g in zip(w, gradient(X,w,y))]

def optimize(X,y, alpha=0.01, eta = 10**-12, iterations = 1000):
    w = random_w(X.shape[1])
    path = list()
    for k in range(iterations):
        SSL = np.sum(squared_loss(X,w,y))
        new_w = update(X,w,y, alpha=alpha)
        new_SSL = np.sum(squared_loss(X,new_w,y))
        w = new_w
        if k>=5 and (new_SSL - SSL <= eta and new_SSL - SSL >= -eta):
            path.append(new_SSL)
            return w, path
        if k % (iterations / 20) == 0:
            path.append(new_SSL)    
    return w, path


In [ ]:
observations = len(df)
X  = df['unemployment_rate'].values.reshape((observations,1))
# X should be always a matrix, never a vector
X = np.column_stack((X,np.ones(observations))) # We add the bias
y  = df['percent_food_desert'].values # y can be a vector

In [ ]:
alpha = 0.048
w, path = optimize(X,y,alpha, eta = 10**-12, iterations = 100)
print ("These are our final coefficients: %s" % w)
print ("Obtained walking on this path of squared loss %s" % path)


In [ ]:
X = df['unemployment_rate'].values
X  = df['unemployment_rate'].values.reshape((observations,1))
# X should be always a matrix, never a vector
X = np.column_stack((X,np.ones(observations))) # We add the bias

In [ ]:
from sklearn.preprocessing import StandardScaler
observations = len(df)
#variables = 'pop2010_in_des
standardization = StandardScaler()
Xst = standardization.fit_transform(X)
original_means = standardization.mean_
originanal_stds = standardization.std_
Xst = np.column_stack((Xst,np.ones(observations)))
y  = df['pop2010_in_des'].values

In [ ]:
alpha = 0.02
w, path = optimize(Xst, y, alpha, eta = 10**-12, iterations = 200)
print ("These are our final standardized coefficients: " + ', \
'.join(map(lambda x: "%0.4f" % x, w)))

In [ ]:
cors = pd.DataFrame(df.corr()["percent_food_desert"]).dropna()
cors.sort_values(by='percent_food_desert').head(10)

In [ ]:
x_cols=['unemployment_rate', 'unemployment_rate', 'cnty_obesity_pct_adj','cnty_inactive_pct','cnty_obesity_pct', 'PCT_18_64','p_hs_edatt','NUMGQTRS'] 
#x_cols=['PCT_18_64','p_hs_edatt','NUMGQTRS','cnty_inactive_pct'] 

df[x_cols]
y = df['percent_food_desert']
X = df[x_cols]
X = sm.add_constant(X)
linear_regression = sm.OLS(y,X)
fitted_model = linear_regression.fit()
fitted_model.summary()

In [ ]:
sns.pairplot(X[['unemployment_rate', 'cnty_obesity_pct_adj','cnty_inactive_pct','cnty_obesity_pct', 'PCT_18_64','p_hs_edatt']].dropna())

In [ ]:
X.info(verbose=True)
data=df[['percent_food_desert','unemployment_rate','cnty_inactive_pct','cnty_obesity_pct', 'PCT_18_64','p_hs_edatt']]

In [ ]:
sns.pairplot(data)

In [ ]:
data.columns.tolist()

In [ ]:
x_cols=['unemployment_rate','cnty_inactive_pct','cnty_obesity_pct', 'PCT_18_64','p_hs_edatt']
df[x_cols]
y = df['percent_food_desert']
X = df[x_cols]
X = sm.add_constant(X)
linear_regression = sm.OLS(y,X)
fitted_model = linear_regression.fit()
fitted_model.summary()

In [ ]:
import statsmodels.formula.api as smf

def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

In [ ]:
x_cols=[ 'percent_food_desert','unemployment_rate', 'unemployment_rate', 'cnty_obesity_pct_adj','cnty_inactive_pct','cnty_obesity_pct', 'PCT_18_64','p_hs_edatt','NUMGQTRS'] 
#x_cols=['PCT_18_64','p_hs_edatt','NUMGQTRS','cnty_inactive_pct'] 

data = df[x_cols]

model = forward_selected(data, 'percent_food_desert')

print model.model.formula

print model.rsquared_adj


In [ ]:
df.columns.tolist()
cols=['opiods_rx_1000', 'pop2010_in_des',
'num_tracts',
'n_food_des',
'n_urban',
'n_rural',
'Rural',
'Urban',
'cnty_obesity_pct',
'cnty_dm_pct',
'cnty_inactive_pct',
'POP2010',
'OHU2010',
'NUMGQTRS',
'HUNVFlag',
'Adolescent_births',
'ABR',
'p_hs_edatt',
'PC_PHYS_R',
'DENTIST_R',
'PSYCH_R',
'PCT_HSPNC',
'PCT_WHITE',
'PCT_BLACK',
'PCT_ASIAN',
'PCT_AMIND_ESK',
'PCT_ISLANDER',
'PCT_MULTI',
'PCT_OTHER',
'PCT_65OVER',
'PCT_18_64',
'PCT_UNDR18',
'PCT_UNDER5',
'unemployment_rate',
'n_hospitals']

In [ ]:
data =df[cols].dropna()
data
# model = forward_selected(data, 'pop2010_in_des')

# print model.model.formula

# print model.rsquared_adj


In [ ]:
from sklearn.cross_validation import cross_val_score, ShuffleSplit
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
 
#Load boston housing dataset as an example
X =data.drop("pop2010_in_des",axis=1)
X=X.values
Y = data["pop2010_in_des"].values
names = data.columns
 
rf = RandomForestRegressor(n_estimators=20, max_depth=4)
scores = []
for i in range(X.shape[1]):
    score = cross_val_score(rf, X[:, i:i+1], Y, scoring="r2", cv=ShuffleSplit(len(X), 3, .3))
    scores.append((round(np.mean(score), 3), names[i]))
print sorted(scores, reverse=True)

In [ ]:
feats=pd.DataFrame(sorted(scores, reverse=True),columns=['importance','feature'])
feats.plot(kind='bar')
feats

In [ ]:

def plot_importances(X_train, importances, f_rank):
    '''
    :param X_train: Dataframe containing predictor variables of training set
    :param importances: List of feature importance, obtained from try_forest
    :param f_rank: Feature ranking, obtained from sort_features()
    :return: None, plot saved as 'Forest_feature_importances.png'
    '''
    plt.title('Feature Importances according to Random Forest')
    plt.bar(range(X_train.shape[1]), importances[indices], color='lightblue', align='center')
    plt.xticks(range(X_train.shape[1]), f_rank, rotation=45)
    plt.xlim([-1, X_train.shape[1]])
    plt.tight_layout()

In [ ]:
sns.barplot(x=feats['feature'][:5],y=feats['importance'][:5])

In [ ]:
sns.barplot(x=feats['feature'][-5:],y=feats['importance'][-5:])

In [ ]:
import statsmodels.formula.api as sm

class LinReg:
    kind='Linear Regression'
    
    def __init__(self, df):
        self.var=len(df.columns)
        self.obs=len(df.index)
        self.df=df

    def model(self,x,y):
        X=x[0]
        for i in range(len(x)-1):
            X=X+'+'+x[i+1]
        mod=y+'~'+X
        return mod
    
    def linreg(self,x, y):
        result=sm.ols(formula=self.model(x,y), data=self.df).fit()
        return result
    

In [ ]:
d=LinReg(df)
x=['n_rural','n_urban']

print d.var
print d.obs
print d.model(x,'pop2010_in_des')
fit=d.linreg(x,'pop2010_in_des')

print fit.summary()
print fit.params
print fit.tvalues
print fit.pvalues
print fit.rsquared
print fit.f_test(np.identity(len(x)+1))



In [ ]:

from numpy import arange,array,ones,asarray,vstack,linalg
from scipy import stats

def liner_regress(x, y, type=1):
    if (type==1) : 
        A = vstack([x, ones(len(x))]).T
        slope, intercept = linalg.lstsq(A, y)[0]
    else:
        slope, intercept, r, p, std_err = stats.linregress(x,y)
        #print 'r value', r_value
        #print  'p_value', p_value
        #print 'standard deviation', std_err

    print 'y = '+str(slope)+' * x + '+str(intercept)
   
    liney = slope*x + intercept
    return liney, slope, intercept 


def _chunks(lst, size):
    for i in xrange(0, len(lst), size):
        yield lst[i:i+size]

def _liner_regression(x, y, chunks_size, reduce_fn):
    ys = [reduce_fn(yy) for yy in _chunks(y, chunks_size)]
    xs = [max(xx) for xx in _chunks(x, chunks_size)]
    liney, slope, intercept = liner_regression(xs, ys)
    liney = slope*x + intercept
    return liney, slope, intercept

def liner_regression_max(x,y, chunks_size):
    return _liner_regression(x, y, chunks_size, max)

def liner_regression_min(x,y, chunks_size):
    return _liner_regression(x, y, chunks_size, min)

liner_regress(df['n_rural'], df['pop2010_in_des'])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas

# For 3d plots. This import is necessary to have 3D plotting below
from mpl_toolkits.mplot3d import Axes3D

# For statistics. Requires statsmodels 5.0 or more
from statsmodels.formula.api import ols
# Analysis of Variance (ANOVA) on linear models
from statsmodels.stats.anova import anova_lm

##############################################################################
# Generate and show the data
x = np.linspace(-5, 5, 21)
# We generate a 2D grid
X, Y = np.meshgrid(x, x)

# To get reproducable values, provide a seed value
np.random.seed(1)

# Z is the elevation of this 2D grid
Z = -5 + 3*X - 0.5*Y + 8 * np.random.normal(size=X.shape)

# Plot the data
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z, cmap=plt.cm.coolwarm,
                       rstride=1, cstride=1)
ax.view_init(20, -120)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

##############################################################################
# Multilinear regression model, calculating fit, P-values, confidence
# intervals etc.

# Convert the data into a Pandas DataFrame to use the formulas framework
# in statsmodels

# First we need to flatten the data: it's 2D layout is not relevent.
X = X.flatten()
Y = Y.flatten()
Z = Z.flatten()

data = pandas.DataFrame({'x': X, 'y': Y, 'z': Z})

# Fit the model
model = ols("z ~ x + y", data).fit()

# Print the summary
print(model.summary())

print("\nRetrieving manually the parameter estimates:")
print(model._results.params)
# should be array([-4.99754526,  3.00250049, -0.50514907])

# Peform analysis of variance on fitted linear model
anova_results = anova_lm(model)

print('\nANOVA results')
print(anova_results)



In [ ]:

from scipy import stats
import numpy
import math

class DataSet :
    def __init__(self, X, Y) :
        self.X = X
        self.Y = Y
        
    def fn(self, function, *args) :
        return function(self.X, self.Y, *args)

def Sss(X, Y):
    return numpy.float64(sum([X[i] * Y[i] - mean(X) * mean(Y) for i in range(0, len(X))]))

def Sxx(X, Y) :
    return Sss(X,X)

def Syy(X, Y) :
    return Sss(Y, Y)

def Sxy(X, Y) :
    return Sss(X, Y)

def mean(X) :
    return (numpy.float64(1.0) / numpy.float64(len(X))) * numpy.float64(sum(X))

def slope(X, Y) :
    return Sss(X, Y) / Sss(X, X)

def intercept(X, Y) :
    return mean(Y) - slope(X,Y) * mean(X)

def expected(X, Y, x) :
    return intercept(X, Y) + slope(X, Y) * x 

def residuals(X, Y) :
    return [Y[i] - expected(X, Y, X[i]) for i in range(0, len(Y))]

def SST(X, Y) :
    return Sss(Y, Y)

def SSR(X, Y) :
    return (slope(X, Y) ** 2.0) * Sss(X, X)

def SSE(X, Y) :
    return SST(X, Y) - SSR(X, Y)

def rsquared(X, Y) :
    return 1 - SSE(X, Y) / SST(X, Y)

def r(X, Y) :
    rVal = math.sqrt(rsquared(X, Y))
    if slope(X, Y) < 0.0 :
        rVal *= -1.0

    return rVal

def variance(X, Y) :
    return SSE(X, Y) / numpy.float64(len(X) - 2)

def interceptError(X, Y) :
    dev = numpy.float64(math.sqrt(variance(X, Y)))
    a   = sum(X[i] * X[i] for i in range(0, len(X)))
    b   = len(X) * Sss(X,X)

    return dev * numpy.float64(math.sqrt(a / b))

def slopeError(X, Y) :
    dev = numpy.float64(math.sqrt(variance(X, Y)))
    return dev / numpy.float64(math.sqrt(Sss(X, X)))

def slopeCI(X, Y, alpha) :
    s = slope(X, Y)
    diff = abs(stats.t.ppf(alpha / 2.0, len(X) - 2)) * slopeError(X, Y)
    return (s - diff, s + diff)

def interceptCI(X, Y, alpha) :
    i = intercept(X, Y)
    diff = abs(stats.t.ppf(alpha / 2.0, len(X) - 2)) * interceptError(X, Y)
    return (i - diff, i + diff)

def slopeHypothesis(X, Y, null, alpha) :
    diff = abs((slope(X, Y) - numpy.float64(null))) / slopeError(X, Y)
    test = abs(stats.t.ppf(alpha / 2.0, len(X) - 2))

    if diff > test :
        return True

    return False

def MSR(X, Y) :
    return SSR(X, Y)

def MSE(X, Y) :
    return SSE(X, Y) / numpy.float64(len(X) - 2)

def ANOVA(X, Y) :
    print "(Source, SS, df, MS, F)"
    print "Regression", SSR(X, Y), " ", 1, " ", MSR(X, Y), " ", MSR(X,Y) / MSE(X,Y)
    print "Error     ", SSE(X, Y), " ", len(X) - 2, " ", MSE(X,Y)
    print "Total     ", SST(X, Y), " ", len(X) - 1

def predictionInterval(X, Y, x, alpha) :
    expect = expected(X, Y, numpy.float64(x))
    tstat  = abs(stats.t.ppf(alpha / 2.0, len(X) - 2))
    dev    = numpy.float64(math.sqrt(variance(X, Y)))
    a      = 1.0 / numpy.float64(len(X))
    b      = ((numpy.float64(x) - mean(X)) ** 2.0) / Sss(X,X)
    diff   = tstat * dev * math.sqrt(1 + a + b)

    return (expect - diff, expect + diff)

def standardResiduals(X, Y) :
    dev = math.sqrt(variance(X,Y))
    return [residuals(X, Y)[i] / (dev * math.sqrt(1 - leverage(X, Y, i))) for i in range(0, len(X))]

def outliers(X, Y) :
    return [i for i in range(0, len(X)) if standardResiduals(X,Y)[i] > 2.0]

def leverage(X, Y, i) :
    return (1.0 / numpy.float64(len(X))) + ((X[i] - mean(X)) ** 2.0) / Sss(X, X)

def influentials(X, Y) :
    threshold = 2.0 * (2.0) / float(len(X))
    return [i for i in range(0, len(X)) if leverage(X,Y,i) > threshold]

In [ ]:
X

In [ ]:

class LinReg(object):
    """
    multivariate linear regression using gradient descent
    """
    def __init__(self, learning_rate = 0.01, iterations = 50, verbose = True, l2 = 0, 
                tolerance = 0, intercept = True):
        """
        :param learning_rate: learning rate constant
        :param iterations: how many epochs
        :param tolerance: the error value in which to stop training
        :param intercept: whether to fit an intercept
        :param verbose: whether to spit out error rates while training
        :param l2: L2 regularization term
        """
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.tolerance = tolerance
        self.intercept = intercept
        self.verbose = verbose
        self.l2 = l2
        self.theta = None
        self.mean = []
        self.std = []

    def fit(self, X, y):
        """
        Gradient descent, loops over theta and updates to
        take steps in direction of steepest decrease of J.
        :return: value of theta that minimizes J(theta) and J_history
        """
        if self.intercept:
            intercept = np.ones((np.shape(X)[0],1))
            X = np.concatenate((intercept, X), 1)
            
        num_examples, num_features = np.shape(X)

        # initialize theta to 1
        self.theta = np.ones(num_features)

        for i in range(self.iterations):
            # make prediction
            predicted = np.dot(X, self.theta.T)
            # update theta with gradient descent
            self.theta = (self.theta * (1 - (self.learning_rate * self.l2))) - self.learning_rate / num_examples * np.dot((predicted - y).T, X)
            # sum of squares cost
            error = predicted - y
            cost = np.sum(error**2) / (2 * num_examples)
            
            if i % 10 == 0 and self.verbose == True:
                print 'iteration:', i
                print 'theta:', self.theta
                print 'cost:', cost
                
            if cost < self.tolerance:
                return self.theta
                break

        return self.theta

    def predict(self, X):
        """
        Make linear prediction based on cost and gradient descent
        :param X: new data to make predictions on
        :return: return prediction
        """
        if self.intercept:
            intercept = np.ones((np.shape(X)[0],1))
            X = np.concatenate((intercept, X), 1)
        
        num_examples, num_features = np.shape(X)
        prediction = []
        for sample in range(num_examples):
            yhat = 0
            for value in range(num_features):
                yhat += X[sample, value] * self.theta[value]
            prediction.append(yhat)
                
        return prediction

def demo():
    # initialize linear regression parameters
    iterations = 2000
    learning_rate = 0.1
    l2 = 0.0001

    linearReg = LinReg(learning_rate = learning_rate, iterations = iterations, verbose = 1, l2 = l2)

    #data = np.genfromtxt('Data/blood_pressure.csv', delimiter = ',', skip_header = 1)
    #X = data[:, 1:]
    #y = data[:, 0]
    y = data["pop2010_in_des"].values

    X =data.drop("pop2010_in_des",axis=1)
    
    # scale data
    max = np.amax(X)
    X /= max
    #print X
    #print y

    # fit the linear reg
    linearReg.fit(X = X, y = y)

    # load testing dataset
    test = np.genfromtxt('Data/blood_pressure.csv', delimiter = ',', skip_header = 1)
    X_test = test[:, 1:]
    y_test = test[:, 0]
    
    max = np.amax(X_test)
    X_test /= max
    #print X_test

    predictions = np.array(linearReg.predict(X_test))

    #print 'correct: ', y_test
    #print 'prediction: ', predictions
    return y_test,predictions

In [ ]:
lst=demo()

In [ ]:
lst[0]